# Database Check
Validate raw + derived tables and provenance for the FINRA + Polygon pipeline.


In [1]:
from darkpool_analysis.config import load_config
from darkpool_analysis.db import get_connection
import pandas as pd

config = load_config()
conn = get_connection(config.db_path)


In [2]:
tables = conn.execute("SHOW TABLES").df()["name"].tolist()
tables


['composite_signal',
 'daily_metrics',
 'finra_otc_weekly_raw',
 'finra_short_daily_all_raw',
 'finra_short_daily_raw',
 'index_constituent_short_agg_daily',
 'lit_direction_daily',
 'options_premium_daily',
 'options_premium_summary',
 'polygon_daily_agg_raw',
 'polygon_equity_trades_raw',
 'polygon_ingestion_state',
 'scanner_daily_metrics']

In [3]:
from IPython.display import display

for table in tables:
    print(table)
    display(conn.execute(f"DESCRIBE {table}").df())
    display(conn.execute(f"SELECT COUNT(*) AS n FROM {table}").df())


composite_signal


,column_name,column_type,null,key,default,extra
0,symbol,VARCHAR,YES,None,None,None
1,date,DATE,YES,None,None,None
2,directional_score,DOUBLE,YES,None,None,None
3,pressure_score,DOUBLE,YES,None,None,None
4,participation_score,DOUBLE,YES,None,None,None
5,composite_score,DOUBLE,YES,None,None,None
6,final_label,VARCHAR,YES,None,None,None
7,data_quality,VARCHAR,YES,None,None,None
8,inference_version,VARCHAR,YES,None,None,None


,n
0,0


daily_metrics


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,symbol,VARCHAR,YES,None,None,None
2,log_buy_sell,DOUBLE,YES,None,None,None
3,short_volume,DOUBLE,YES,None,None,None
4,short_exempt_volume,DOUBLE,YES,None,None,None
5,short_total_volume,DOUBLE,YES,None,None,None
6,short_buy_volume,DOUBLE,YES,None,None,None
7,short_sell_volume,DOUBLE,YES,None,None,None
8,short_ratio,DOUBLE,YES,None,None,None
9,short_ratio_z,DOUBLE,YES,None,None,None


...

,column_name,column_type,null,key,default,extra
0,symbol,VARCHAR,NO,PRI,None,None
1,trade_date,DATE,NO,PRI,None,None
2,data_source,VARCHAR,NO,PRI,None,None
3,fetch_timestamp,TIMESTAMP,YES,None,None,None
4,record_count,INTEGER,YES,None,None,None
5,status,VARCHAR,YES,None,None,None


,n
0,19422


scanner_daily_metrics


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,symbol,VARCHAR,YES,None,None,None
2,short_volume,DOUBLE,YES,None,None,None
3,short_exempt_volume,DOUBLE,YES,None,None,None
4,total_volume,DOUBLE,YES,None,None,None
5,short_total_volume,DOUBLE,YES,None,None,None
6,short_buy_volume,DOUBLE,YES,None,None,None
7,short_sell_volume,DOUBLE,YES,None,None,None
8,short_ratio,DOUBLE,YES,None,None,None
9,short_ratio_z,DOUBLE,YES,None,None,None


,n
0,0


In [4]:
sample_symbol = "META"
start_date = min(config.target_dates)
end_date = max(config.target_dates)
sample_symbol, start_date, end_date


('META', datetime.date(2025, 11, 18), datetime.date(2025, 12, 31))

## Raw Tables Samples


In [5]:
conn.execute("""
SELECT * FROM finra_otc_weekly_raw
WHERE symbol = ?
ORDER BY week_start_date DESC
LIMIT 5
""", [sample_symbol]).df()


,symbol,week_start_date,off_exchange_volume,trade_count,tier_identifier,tier_description,issue_name,market_participant_name,mpid,last_update_date,source_file
0,META,2025-12-08,35245177.0,1047081.0,T1,NMS Tier 1,"Meta Platforms, Inc. Class A Common Stock",AGGREGATED,None,2025-12-29,None
1,META,2025-12-01,41980483.0,1196465.0,T1,NMS Tier 1,"Meta Platforms, Inc. Class A Common Stock",AGGREGATED,None,2025-12-22,None
2,META,2025-11-24,35889734.0,1041923.0,T1,NMS Tier 1,"Meta Platforms, Inc. Class A Common Stock",AGGREGATED,None,2025-12-15,None
3,META,2025-11-17,56021686.0,1712198.0,T1,NMS Tier 1,"Meta Platforms, Inc. Class A Common Stock",AGGREGATED,None,2025-12-08,None
4,META,2025-11-10,49886574.0,1684716.0,T1,NMS Tier 1,"Meta Platforms, Inc. Class A Common Stock",AGGREGATED,None,2025-12-01,None


In [6]:
conn.execute("""
SELECT * FROM finra_short_daily_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,trade_date,short_volume,short_exempt_volume,total_volume,market,source,source_file
0,META,2025-12-31,1273796.0,10784.0,3126429.0,"B,Q,N",CNMS,CNMSshvol20251231.txt
1,META,2025-12-30,1313715.0,8587.0,3974561.0,"B,Q,N",CNMS,CNMSshvol20251230.txt
2,META,2025-12-29,1274723.0,6196.0,3534482.0,"B,Q,N",CNMS,CNMSshvol20251229.txt
3,META,2025-12-26,1654781.0,6021.0,3071365.0,"B,Q,N",CNMS,CNMSshvol20251226.txt
4,META,2025-12-24,1446982.0,9840.0,2403992.0,"B,Q,N",CNMS,CNMSshvol20251224.txt


In [7]:
conn.execute("""
SELECT * FROM polygon_daily_agg_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,trade_date,open,high,low,close,vwap,volume,fetch_timestamp
0,META,2025-12-31,664.75,665.0000,659.4400,660.09,661.6454,7940359.0,2026-01-02 08:26:09.034170
1,META,2025-12-30,658.69,672.2200,657.8400,665.95,666.6166,9437217.0,2026-01-02 08:28:05.904066
2,META,2025-12-29,658.01,660.2500,654.3900,658.69,657.9425,8506205.0,2026-01-02 08:29:59.650636
3,META,2025-12-26,668.06,668.9499,661.3201,663.29,663.8761,7133598.0,2026-01-02 08:31:52.410202
4,META,2025-12-24,662.53,668.1800,662.2000,667.55,666.4190,5627512.0,2026-01-02 08:33:51.161955


In [8]:
conn.execute("""
SELECT * FROM polygon_equity_trades_raw
WHERE symbol = ? AND timestamp::DATE BETWEEN ? AND ?
ORDER BY timestamp DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,timestamp,price,size,bid,ask,data_source
0,META,2025-12-31 23:50:00,659.2534,7562.0,659.25,659.2500,minute
1,META,2025-12-31 23:49:00,659.3416,651.0,659.30,659.3500,minute
2,META,2025-12-31 23:48:00,659.3146,367.0,659.30,659.3300,minute
3,META,2025-12-31 23:43:00,659.4788,85.0,659.47,659.4700,minute
4,META,2025-12-31 23:41:00,659.4903,504.0,659.49,659.4903,minute


## Derived Tables Samples


In [9]:
conn.execute("""
SELECT * FROM lit_direction_daily
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell,classification_method,lit_coverage_pct,inference_version
0,META,2025-12-31,755897.0,407182.0,0.649910,0.618645,NBBO,17.7340,PhaseA_v1
1,META,2025-12-30,694172.0,520056.0,0.571698,0.288783,NBBO,20.0495,PhaseA_v1
2,META,2025-12-29,1153682.0,594360.0,0.659985,0.663229,NBBO,22.3881,PhaseA_v1
3,META,2025-12-26,943747.0,587877.0,0.616174,0.473340,NBBO,16.7901,PhaseA_v1
4,META,2025-12-24,1595685.0,969899.0,0.621958,0.497866,NBBO,20.9302,PhaseA_v1


In [10]:
conn.execute("""
SELECT * FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 10
""", [sample_symbol, start_date, end_date]).df()


,date,symbol,log_buy_sell,short_volume,short_exempt_volume,short_total_volume,short_buy_volume,short_sell_volume,short_ratio,short_ratio_z,...,accumulation_score_display,confidence,data_quality,has_otc,has_short,has_lit,has_price,pressure_context_label,inference_version,accumulation_short_z_source
0,2025-12-31,META,0.618645,1273796.0,10784.0,3126429.0,1273796.0,1852633.0,0.407428,-0.224176,...,52.028213,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
1,2025-12-30,META,0.288783,1313715.0,8587.0,3974561.0,1313715.0,2660846.0,0.330531,-1.019352,...,44.937750,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
2,2025-12-29,META,0.663229,1274723.0,6196.0,3534482.0,1274723.0,2259759.0,0.360653,-0.758616,...,49.242970,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
3,2025-12-26,META,0.473340,1654781.0,6021.0,3071365.0,1654781.0,1416584.0,0.538777,1.036239,...,65.477864,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
4,2025-12-24,META,0.497866,1446982.0,9840.0,2403992.0,1446982.0,957010.0,0.601908,1.690772,...,73.905072,0.7,PRE_OTC,True,True,True,True,Accumulating,PhaseA_v1,short_buy_sell_ratio_z
5,2025-12-23,META,0.507214,1584573.0,6623.0,3243425.0,1584573.0,1658852.0,0.488549,0.571500,...,62.242272,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
6,2025-12-22,META,0.187612,3476472.0,20881.0,6254992.0,3476472.0,2778520.0,0.555792,1.320762,...,67.058388,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
7,2025-12-19,META,-0.100580,3866204.0,5850.0,7573624.0,3866204.0,3707420.0,0.510483,0.967087,...,57.323418,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
8,2025-12-18,META,-0.357127,2566873.0,4992.0,7101681.0,2566873.0,4534808.0,0.361446,-0.663885,...,40.502905,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
9,2025-12-17,META,0.383652,2253885.0,5733.0,4850445.0,2253885.0,2596560.0,0.464676,0.534912,...,59.096463,0.7,PRE_OTC,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z


In [11]:
conn.execute("""
SELECT index_symbol, trade_date, coverage_count, expected_constituent_count, coverage_pct
FROM index_constituent_short_agg_daily
ORDER BY trade_date DESC
LIMIT 5
""").df()


,index_symbol,trade_date,coverage_count,expected_constituent_count,coverage_pct


## Coverage and Provenance Checks


In [12]:
conn.execute("""
SELECT symbol, COUNT(*) AS rows
FROM daily_metrics
WHERE date BETWEEN ? AND ?
GROUP BY symbol
ORDER BY symbol
""", [start_date, end_date]).df()


,symbol,rows
0,AAPL,30
1,AMZN,30
2,AVGO,30
3,EEM,30
4,EFA,30
5,EWJ,30
6,FXI,30
7,GDX,30
8,GLD,30
9,GOOGL,30


In [13]:
conn.execute("""
SELECT date, symbol, otc_off_exchange_volume, otc_week_used, data_quality
FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
""", [sample_symbol, start_date, end_date]).df()


,date,symbol,otc_off_exchange_volume,otc_week_used,data_quality
0,2025-12-31,META,35245177.0,2025-12-08,PRE_OTC
1,2025-12-30,META,35245177.0,2025-12-08,PRE_OTC
2,2025-12-29,META,35245177.0,2025-12-08,PRE_OTC
3,2025-12-26,META,35245177.0,2025-12-08,PRE_OTC
4,2025-12-24,META,35245177.0,2025-12-08,PRE_OTC
5,2025-12-23,META,35245177.0,2025-12-08,PRE_OTC
6,2025-12-22,META,35245177.0,2025-12-08,PRE_OTC
7,2025-12-19,META,35245177.0,2025-12-08,PRE_OTC
8,2025-12-18,META,35245177.0,2025-12-08,PRE_OTC
9,2025-12-17,META,35245177.0,2025-12-08,PRE_OTC


## Sanity Checks


In [14]:
conn.execute("""
SELECT symbol, date, lit_buy_volume, lit_sell_volume, log_buy_sell
FROM lit_direction_daily
WHERE (lit_buy_volume <= 0 OR lit_sell_volume <= 0)
  AND log_buy_sell IS NOT NULL
LIMIT 10
""").df()


,symbol,date,lit_buy_volume,lit_sell_volume,log_buy_sell


In [15]:
conn.execute("""
SELECT symbol, date, short_ratio, short_ratio_denominator_type
FROM daily_metrics
WHERE short_ratio_denominator_type IS NOT NULL
  AND (short_ratio < 0 OR short_ratio > 1)
LIMIT 10
""").df()


,symbol,date,short_ratio,short_ratio_denominator_type


In [16]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM daily_metrics
GROUP BY symbol, date
HAVING n > 1
""").df()


,symbol,date,n


In [17]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM lit_direction_daily
GROUP BY symbol, date
HAVING n > 1
""").df()


,symbol,date,n


## Lit Data Diagnostics
Checking lit_direction_daily and daily_metrics for lit flow issues.

In [18]:
# 1. Check date range in lit_direction_daily (overall)
print("=== Date range in lit_direction_daily ===")
display(conn.execute("""
    SELECT MIN(date) as min_date, MAX(date) as max_date, COUNT(*) as total_rows
    FROM lit_direction_daily
""").df())

=== Date range in lit_direction_daily ===


,min_date,max_date,total_rows
0,2025-01-03,2025-12-31,9711


In [19]:
# 2. Check which symbols have lit data
print("=== Symbols with lit data (lit_direction_daily) ===")
display(conn.execute("""
    SELECT symbol, COUNT(*) as rows, MIN(date) as min_date, MAX(date) as max_date
    FROM lit_direction_daily
    GROUP BY symbol
    ORDER BY rows DESC
""").df())

=== Symbols with lit data (lit_direction_daily) ===


,symbol,rows,min_date,max_date
0,XLV,249,2025-01-03,2025-12-31
1,KRE,249,2025-01-03,2025-12-31
2,EEM,249,2025-01-03,2025-12-31
3,TSLA,249,2025-01-03,2025-12-31
4,SLV,249,2025-01-03,2025-12-31
5,VIXY,249,2025-01-03,2025-12-31
6,XLC,249,2025-01-03,2025-12-31
7,SPY,249,2025-01-03,2025-12-31
8,UUP,249,2025-01-03,2025-12-31
9,XLF,249,2025-01-03,2025-12-31


In [20]:
# 3. Check lit_direction_daily for sample symbols (AAPL, SPY, XLF)
print("=== Lit data for sample symbols ===")
for sym in ["AAPL", "SPY", "XLF"]:
    print(f"\n--- {sym} ---")
    display(conn.execute("""
        SELECT date, lit_buy_volume, lit_sell_volume, lit_buy_ratio, log_buy_sell
        FROM lit_direction_daily
        WHERE symbol = ?
        ORDER BY date DESC LIMIT 10
    """, [sym]).df())

=== Lit data for sample symbols ===

--- AAPL ---


,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell
0,2025-12-31,1816485.0,916367.0,0.664685,0.684242
1,2025-12-30,2563928.0,988953.0,0.721648,0.952649
2,2025-12-29,1179023.0,1282827.0,0.478917,-0.084380
3,2025-12-26,1350434.0,1656807.0,0.449061,-0.204466
4,2025-12-24,1467913.0,2793750.0,0.344446,-0.643543
5,2025-12-23,2370102.0,2495485.0,0.487115,-0.051550
6,2025-12-22,1638736.0,2491628.0,0.396753,-0.419011
7,2025-12-19,8887468.0,7251014.0,0.550700,0.203501
8,2025-12-18,4038127.0,4693237.0,0.462485,-0.150342
9,2025-12-17,5365043.0,1452582.0,0.786937,1.306562



--- SPY ---


,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell
0,2025-12-31,13563589.0,11695601.0,0.536976,0.148176
1,2025-12-30,7008477.0,7276387.0,0.490623,-0.037514
2,2025-12-29,18712796.0,3971530.0,0.824922,1.550056
3,2025-12-26,7614539.0,3329265.0,0.695785,0.827308
4,2025-12-24,5605172.0,6186374.0,0.475355,-0.098659
5,2025-12-23,11886729.0,8260965.0,0.589980,0.363881
6,2025-12-22,6452749.0,7575170.0,0.459993,-0.160370
7,2025-12-19,10314654.0,28008110.0,0.269152,-0.998929
8,2025-12-18,12176484.0,21262399.0,0.364141,-0.557434
9,2025-12-17,29902909.0,10263568.0,0.744474,1.069355



--- XLF ---


,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell
0,2025-12-31,2608336.0,2810567.0,0.481340,-0.074674
1,2025-12-30,1865673.0,1169252.0,0.614734,0.467258
2,2025-12-29,5301542.0,1645544.0,0.763132,1.169927
3,2025-12-26,4348267.0,922843.0,0.824924,1.550074
4,2025-12-24,2138070.0,405787.0,0.840484,1.661830
5,2025-12-23,2703855.0,2005009.0,0.574205,0.299030
6,2025-12-22,9411513.0,3272774.0,0.741982,1.056296
7,2025-12-19,5291824.0,1915355.0,0.734243,1.016260
8,2025-12-18,3472610.0,2265492.0,0.605184,0.427115
9,2025-12-17,4893274.0,3670702.0,0.571379,0.287479


In [21]:
# 4. Check daily_metrics lit columns for sample symbols
print("=== Daily_metrics lit columns for sample symbols ===")
for sym in ["AAPL", "SPY", "XLF"]:
    print(f"\n--- {sym} ---")
    display(conn.execute("""
        SELECT date, lit_flow_imbalance, lit_buy_ratio, lit_buy_volume, lit_sell_volume
        FROM daily_metrics
        WHERE symbol = ?
        ORDER BY date DESC LIMIT 10
    """, [sym]).df())

=== Daily_metrics lit columns for sample symbols ===

--- AAPL ---


,date,lit_flow_imbalance,lit_buy_ratio,lit_buy_volume,lit_sell_volume
0,2025-12-31,0.329369,0.664685,1816485.0,916367.0
1,2025-12-30,0.443295,0.721648,2563928.0,988953.0
2,2025-12-29,-0.042165,0.478917,1179023.0,1282827.0
3,2025-12-26,-0.101878,0.449061,1350434.0,1656807.0
4,2025-12-24,-0.311108,0.344446,1467913.0,2793750.0
5,2025-12-23,-0.025769,0.487115,2370102.0,2495485.0
6,2025-12-22,-0.206493,0.396753,1638736.0,2491628.0
7,2025-12-19,0.101401,0.550700,8887468.0,7251014.0
8,2025-12-18,-0.075030,0.462485,4038127.0,4693237.0
9,2025-12-17,0.573874,0.786937,5365043.0,1452582.0



--- SPY ---


,date,lit_flow_imbalance,lit_buy_ratio,lit_buy_volume,lit_sell_volume
0,2025-12-31,0.073953,0.536976,13563589.0,11695601.0
1,2025-12-30,-0.018755,0.490623,7008477.0,7276387.0
2,2025-12-29,0.649844,0.824922,18712796.0,3971530.0
3,2025-12-26,0.391571,0.695785,7614539.0,3329265.0
4,2025-12-24,-0.049290,0.475355,5605172.0,6186374.0
5,2025-12-23,0.179959,0.589980,11886729.0,8260965.0
6,2025-12-22,-0.080013,0.459993,6452749.0,7575170.0
7,2025-12-19,-0.461696,0.269152,10314654.0,28008110.0
8,2025-12-18,-0.271717,0.364141,12176484.0,21262399.0
9,2025-12-17,0.488949,0.744474,29902909.0,10263568.0



--- XLF ---


,date,lit_flow_imbalance,lit_buy_ratio,lit_buy_volume,lit_sell_volume
0,2025-12-31,-0.037320,0.481340,2608336.0,2810567.0
1,2025-12-30,0.229469,0.614734,1865673.0,1169252.0
2,2025-12-29,0.526264,0.763132,5301542.0,1645544.0
3,2025-12-26,0.649849,0.824924,4348267.0,922843.0
4,2025-12-24,0.680967,0.840484,2138070.0,405787.0
5,2025-12-23,0.148411,0.574205,2703855.0,2005009.0
6,2025-12-22,0.483964,0.741982,9411513.0,3272774.0
7,2025-12-19,0.468487,0.734243,5291824.0,1915355.0
8,2025-12-18,0.210369,0.605184,3472610.0,2265492.0
9,2025-12-17,0.142758,0.571379,4893274.0,3670702.0


In [22]:
# 5. Compare config target_dates with DB date ranges
print("=== Current config target_dates ===")
print(f"Min: {min(config.target_dates)}, Max: {max(config.target_dates)}")
print(f"Count: {len(config.target_dates)}")

print("\n=== Overlap check: lit_direction_daily dates within target range ===")
display(conn.execute("""
    SELECT COUNT(*) as matching_rows
    FROM lit_direction_daily
    WHERE date >= ? AND date <= ?
""", [min(config.target_dates), max(config.target_dates)]).df())

=== Current config target_dates ===
Min: 2025-11-18, Max: 2025-12-31
Count: 30

=== Overlap check: lit_direction_daily dates within target range ===


,matching_rows
0,1170


In [23]:
# 6. Check polygon_ingestion_state (caching)
print("=== Polygon ingestion state (cached trades) ===")
display(conn.execute("""
    SELECT symbol, data_source, COUNT(*) as cached_dates, 
           MIN(trade_date) as min_date, MAX(trade_date) as max_date
    FROM polygon_ingestion_state
    GROUP BY symbol, data_source
    ORDER BY symbol
""").df())

=== Polygon ingestion state (cached trades) ===


,symbol,data_source,cached_dates,min_date,max_date
0,AAPL,minute,249,2025-01-03,2025-12-31
1,AAPL,daily,249,2025-01-03,2025-12-31
2,AMZN,minute,249,2025-01-03,2025-12-31
3,AMZN,daily,249,2025-01-03,2025-12-31
4,AVGO,daily,249,2025-01-03,2025-12-31
...,...,...,...,...,...
73,XLU,minute,249,2025-01-03,2025-12-31
74,XLV,minute,249,2025-01-03,2025-12-31
75,XLV,daily,249,2025-01-03,2025-12-31
76,XLY,minute,249,2025-01-03,2025-12-31


In [24]:
conn.close()
